# Proyecto Módulo 2
## Aplicaciones del Machine Learning
### Camilo Andres Gomez Vargas
### Santiago Ortiz Pérez

### Cargamos las librerías ...

In [11]:
#!pip3 install -r requirements.txt
#!pip3 install tomotopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 4.2 MB/s eta 0:00:00
Reason for being yanked: Regression: https://github.com/aio-libs/aiohttp/issues/10617
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.2/63.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 10.5 MB/s eta 0:00:0

In [3]:
#from utils import *
import pandas as pd
import numpy as np
import tomotopy as tp
import seaborn as sns

import re
import spacy
import os
import matplotlib.pyplot as plt
from wordcloud import WordCloud

import tomotopy as tp
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import numpy as np

#from sklearn.model_selection import KFold

Agregamos el NLP en inglés de Spacy

In [16]:
# ! python -m spacy download en_core_web_sm

Cargamos los datos

In [39]:
data  = pd.read_csv('data_clean.csv')
data.head(10)

,title,overview,genres,genre,genre code
0,Father of the Bride Part II,Just when George Banks has recovered from his ...,"[{'id': 35, 'name': 'Comedy'}]",Comedy,1
1,Othello,The evil Iago pretends to be friend of Othello...,"[{'id': 18, 'name': 'Drama'}]",Drama,2
2,Dead Man Walking,A justice drama based on a true story about a ...,"[{'id': 18, 'name': 'Drama'}]",Drama,2
3,"Cry, the Beloved Country",A South-African preacher goes to search for hi...,"[{'id': 18, 'name': 'Drama'}]",Drama,2
4,Georgia,"Sadie looks up to her older sister Georgia, a ...","[{'id': 18, 'name': 'Drama'}]",Drama,2
5,Don't Be a Menace to South Central While Drink...,When Ashtray (Shawn Wayans) moves to South Cen...,"[{'id': 35, 'name': 'Comedy'}]",Comedy,1
6,Bio-Dome,Bud and Doyle are two losers who are doing not...,"[{'id': 35, 'name': 'Comedy'}]",Comedy,1
7,Friday,Craig and Smokey are two guys in Los Angeles h...,"[{'id': 35, 'name': 'Comedy'}]",Comedy,1
8,Nico Icon,"A look into the many lives of Christa Päffgen,...","[{'id': 99, 'name': 'Documentary'}]",Documentary,3
9,Black Sheep,When dignified Albert Donnelly runs for Govern...,"[{'id': 35, 'name': 'Comedy'}]",Comedy,1


Definimos una función auxiliar para preprocesar el texto antes del entrenamiento del SLDA.

In [40]:
def preprocesar_texto(texto):
    """
    Función genérica que realiza el preprocesamiento de texto para modelos LDA en español.
    Incluye POS tagging, lematización y filtrado según mejores prácticas.
    Devuelve una lista de tokens procesados.
    """

    # 1. Eliminación de URLs y correos electrónicos
    texto = re.sub(r'http\S+', '', texto)
    texto = re.sub(r'\S+@\S+', '', texto)

    # 2. Normalización de espacios y caracteres especiales
    texto = re.sub(r'\s+', ' ', texto).strip().lower()

    # 3. Procesamiento con SpaCy (tokenización, POS tagging, lematización)
    doc = nlp(texto)

    tokens_procesados = []

    for token in doc:
        # 4. Filtrar stopwords, puntuación y números
        if token.is_stop or token.is_punct or token.like_num:
            continue

        # 5. Filtrar por POS: mantener sustantivos, adjetivos y verbos en infinitivo
        if token.pos_ not in ['NOUN', 'ADJ', 'VERB']:
            continue

        # 6. Lematización y limpieza final
        lemma = token.lemma_.lower().strip()

        # 7. Filtrar lemas cortos y caracteres no deseados
        if len(lemma) < 3 or not re.match(r'^[a-záéíóúñü]+$', lemma):
            continue

        tokens_procesados.append(lemma)

    return tokens_procesados

Aplicamos la funcion sobre la columna overview para tener las sinopsis procesadas para poder entrenar el modelo

In [41]:
nlp = spacy.load("en_core_web_sm")

data['texto_procesado'] = data['overview'].apply(preprocesar_texto)
data.head(5)

,title,overview,genres,genre,genre code,texto_procesado
0,Father of the Bride Part II,Just when George Banks has recovered from his ...,"[{'id': 35, 'name': 'Comedy'}]",Comedy,1,"[bank, recover, daughter, wedding, receive, ne..."
1,Othello,The evil Iago pretends to be friend of Othello...,"[{'id': 18, 'name': 'Drama'}]",Drama,2,"[evil, iago, pretend, friend, order, manipulat..."
2,Dead Man Walking,A justice drama based on a true story about a ...,"[{'id': 18, 'name': 'Drama'}]",Drama,2,"[justice, drama, base, true, story, man, death..."
3,"Cry, the Beloved Country",A South-African preacher goes to search for hi...,"[{'id': 18, 'name': 'Drama'}]",Drama,2,"[south, african, preacher, search, wayward, so..."
4,Georgia,"Sadie looks up to her older sister Georgia, a ...","[{'id': 18, 'name': 'Drama'}]",Drama,2,"[sadie, look, old, sister, successful, folk, s..."


Para este proyecto hemos decidido hacer un clasificador para el género Drama, el cual se encuentra codificado con el número 2 dentro de nuestra base. Hemos decidido usar este genero ya que es el que más presencia tiene dentro de la base.

In [42]:
data['genre code'] = data['genre code'].apply(lambda x: 1 if x == 2 else 0) # Clasificamos para el género Drama


Guardamos los datos listos para entrenar el modelo

In [18]:
#data.to_csv('data_procesada.csv', index=False)
#data.head(10)

### Entrenamiento del Modelo

Empezaremos haciendo una optimización del hiperparametro de la cantidad de temas por validación cruzada usando k=5 folds, la métrica que usaremos de guía es ROC AUC.

In [43]:
# Parámetros
k_values = range(10, 21)  # Valores de k a evaluar
n_folds = 5               # Número de folds
n_iters = 50              # Número de iteraciones de entrenamiento

In [44]:
docs = list(data.texto_procesado) # Los documentos
labels = list(data['genre code']) # Las Etiquetas

In [45]:
# Guardar resultados de AUC para cada k
auc_resultados = {}

# Validación cruzada
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

for k in k_values:
    print(f"\nEvaluando k = {k}")
    auc_folds = []

    for fold_idx, (train_idx, test_idx) in enumerate(skf.split(docs, labels)):
        # Crear modelo SLDA
        model = tp.SLDAModel(k=k, seed=123, vars='b')

        # Añadir documentos de entrenamiento
        for idx in train_idx:
            model.add_doc(words=docs[idx], y=[labels[idx]])

        # Entrenar el modelo
        model.train(n_iters)

        # Predecir en los documentos de validación
        y_true = []
        y_scores = []

        for idx in test_idx:
            if docs[idx] != []:
              doc_words = docs[idx]
              label = labels[idx]
              y_true.append(label)

              #print(doc_words)

              # Inferir el documento en el modelo entrenado
              test_doc = model.make_doc(doc_words)
              model.infer(test_doc)

              # Obtener la probabilidad estimada de y=1
              prob = model.estimate(test_doc)
              y_scores.append(prob)

        # Calcular AUC para este fold
        auc = roc_auc_score(y_true, y_scores)
        auc_folds.append(auc)
        print(f"  Fold {fold_idx+1} - AUC: {auc:.4f}")

    # Promediar AUC en los 5 folds
    auc_medio = np.mean(auc_folds)
    auc_resultados[k] = auc_medio
    print(f"Promedio AUC para k={k}: {auc_medio:.4f}")

# Mostrar los resultados y el mejor k
mejor_k = max(auc_resultados, key=auc_resultados.get)
print("\nResultados de AUC por k:")
for k in sorted(auc_resultados):
    print(f"  k={k}: AUC={auc_resultados[k]:.4f}")
print(f"\nMejor valor de k: {mejor_k} con AUC={auc_resultados[mejor_k]:.4f}")


Evaluando k = 10


<ipython-input-45-9154d0d5f151>:20: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  model.train(n_iters)


  Fold 1 - AUC: 0.7369


<ipython-input-45-9154d0d5f151>:20: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  model.train(n_iters)


  Fold 2 - AUC: 0.7468


...

  Fold 3 - AUC: 0.7160


<ipython-input-45-9154d0d5f151>:20: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  model.train(n_iters)


  Fold 4 - AUC: 0.7574


<ipython-input-45-9154d0d5f151>:20: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  model.train(n_iters)


  Fold 5 - AUC: 0.7559
Promedio AUC para k=20: 0.7443

Resultados de AUC por k:
  k=10: AUC=0.7295
  k=11: AUC=0.7293
  k=12: AUC=0.7326
  k=13: AUC=0.7322
  k=14: AUC=0.7399
  k=15: AUC=0.7361
  k=16: AUC=0.7350
  k=17: AUC=0.7410
  k=18: AUC=0.7428
  k=19: AUC=0.7467
  k=20: AUC=0.7443

Mejor valor de k: 19 con AUC=0.7467


La validación cruzada muestra que el mejor valor de k es 19 temas con un AUC=0.7467



## K-folds

Repetimos el proceso ahora con k=10 folds




In [46]:
# Parámetros
k_values = range(10, 21)  # Valores de k a evaluar
n_folds = 10               # Número de folds
n_iters = 50              # Número de iteraciones de entrenamiento

# Guardar resultados de AUC para cada k
auc_resultados = {}

# Validación cruzada
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

for k in k_values:
    print(f"\nEvaluando k = {k}")
    auc_folds = []

    for fold_idx, (train_idx, test_idx) in enumerate(skf.split(docs, labels)):
        # Crear modelo SLDA
        model = tp.SLDAModel(k=k, seed=123, vars='b')

        # Añadir documentos de entrenamiento
        for idx in train_idx:
            model.add_doc(words=docs[idx], y=[labels[idx]])

        # Entrenar el modelo
        model.train(n_iters)

        # Predecir en los documentos de validación
        y_true = []
        y_scores = []

        for idx in test_idx:
            if docs[idx] != []:
              doc_words = docs[idx]
              label = labels[idx]
              y_true.append(label)

              #print(doc_words)

              # Inferir el documento en el modelo entrenado
              test_doc = model.make_doc(doc_words)
              model.infer(test_doc)

              # Obtener la probabilidad estimada de y=1
              prob = model.estimate(test_doc)
              y_scores.append(prob)

        # Calcular AUC para este fold
        auc = roc_auc_score(y_true, y_scores)
        auc_folds.append(auc)
        print(f"  Fold {fold_idx+1} - AUC: {auc:.4f}")

    # Promediar AUC en los 5 folds
    auc_medio = np.mean(auc_folds)
    auc_resultados[k] = auc_medio
    print(f"Promedio AUC para k={k}: {auc_medio:.4f}")

# Mostrar los resultados y el mejor k
mejor_k = max(auc_resultados, key=auc_resultados.get)
print("\nResultados de AUC por k:")
for k in sorted(auc_resultados):
    print(f"  k={k}: AUC={auc_resultados[k]:.4f}")
print(f"\nMejor valor de k: {mejor_k} con AUC={auc_resultados[mejor_k]:.4f}")


Evaluando k = 10


<ipython-input-46-44676af2443f>:25: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  model.train(n_iters)


  Fold 1 - AUC: 0.7194


<ipython-input-46-44676af2443f>:25: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  model.train(n_iters)


  Fold 2 - AUC: 0.7123


...

  Fold 8 - AUC: 0.7638


<ipython-input-46-44676af2443f>:25: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  model.train(n_iters)


  Fold 9 - AUC: 0.7607


<ipython-input-46-44676af2443f>:25: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  model.train(n_iters)


  Fold 10 - AUC: 0.7276
Promedio AUC para k=20: 0.7442

Resultados de AUC por k:
  k=10: AUC=0.7330
  k=11: AUC=0.7447
  k=12: AUC=0.7423
  k=13: AUC=0.7382
  k=14: AUC=0.7428
  k=15: AUC=0.7446
  k=16: AUC=0.7407
  k=17: AUC=0.7404
  k=18: AUC=0.7442
  k=19: AUC=0.7438
  k=20: AUC=0.7442

Mejor valor de k: 11 con AUC=0.7447


Con k=10 folds nos recomienda usar 11 temas.

### Analisis de Resultados

Por la recomendación de la validación cruzada usando k=5 y k=10 folds decidimos usar un modelo con 11 temas.

In [47]:
modelo_final = tp.SLDAModel(k=11, seed=123,vars='b')


# Añadir documentos con sus etiquetas de sentimiento correspondientes
for i in range(len(docs)):
    modelo_final.add_doc(words = docs[i], y=[labels[i]])

# Entrenar el modelo
modelo_final.train(100)

<ipython-input-47-8297b657db6a>:9: RuntimeWarning: The training result may differ even with fixed seed if `workers` != 1.
  modelo_final.train(100)


### Interpretación del vector de regresión

In [51]:
# Obtener los coeficientes de regresión para la respuesta binaria (log-odds)
coeficientes_regresion_binario = modelo_final.get_regression_coef(0)
print("\nCoeficientes de Regresión para Sentimiento (Log-Odds):")
for i, coef in enumerate(coeficientes_regresion_binario):
    print(f"Tema #{i}: {coef:.4f}")


Coeficientes de Regresión para Sentimiento (Log-Odds):
Tema #0: 4.2624
Tema #1: -3.6531
Tema #2: 3.8736
Tema #3: -0.8699
Tema #4: -1.9062
Tema #5: -3.9992
Tema #6: 0.0252
Tema #7: -1.8815
Tema #8: -2.0482
Tema #9: -0.8353
Tema #10: -3.4998


### Nuevas Sinopsis

In [52]:
drama_movies = {
    "Sirât": {
        "overview": (
            "A father and his son arrive at a rave deep in the mountains of southern Morocco. "
            "They’re searching for Mar — daughter and sister — who vanished months ago at one of these endless, sleepless parties. "
            "Surrounded by electronic music and a raw, unfamiliar sense of freedom, they hand out her photo again and again. "
            "Hope is fading but they push through and follow a group of ravers heading to one last party in the desert. "
            "As they venture deeper into the burning wilderness, the journey forces them to confront their own limits."
        ),
        "genre_code": 1
    },
    "The Girl with the Needle": {
        "overview": (
            "Copenhagen 1919: A young worker finds herself unemployed and pregnant. "
            "She meets Dagmar, who runs an underground adoption agency. "
            "A strong connection grows but her world shatters when she stumbles on the shocking truth behind her work."
        ),
        "genre_code": 1
    },
    "The Drama": {
        "overview": (
            "A couple, in the days leading up to their wedding, faces a crisis when unexpected revelations derail what one of them thought they knew about the other."
        ),
        "genre_code": 1
    }
}
df_new = pd.DataFrame.from_dict(drama_movies, orient='index')
df_new


nlp = spacy.load("en_core_web_sm")

df_new['texto_procesado'] = df_new['overview'].apply(preprocesar_texto)



In [53]:
df_new

,overview,genre_code,texto_procesado
Sirât,A father and his son arrive at a rave deep in ...,1,"[father, son, arrive, rave, mountain, southern..."
The Girl with the Needle,Copenhagen 1919: A young worker finds herself ...,1,"[young, worker, find, unemployed, pregnant, me..."
The Drama,"A couple, in the days leading up to their wedd...",1,"[couple, day, lead, wedding, face, crisis, une..."


In [54]:
for i in range(0,3):

  doc_inst = modelo_final.make_doc(df_new['texto_procesado'][i])
  topic_dist, _ = modelo_final.infer(doc_inst)
  print("Distribución de los temas", topic_dist)

  prob_dist = modelo_final.estimate(doc_inst)
  print("P(Y=1|texto) con Y=1 -> Positivo", prob_dist)

Distribución de los temas [0.22314441 0.03505515 0.03432328 0.19234486 0.01154179 0.12608656
 0.15446094 0.00966374 0.07716232 0.00943771 0.12677926]
P(Y=1|texto) con Y=1 -> Positivo [0.4225638]
Distribución de los temas [0.15102118 0.10331837 0.02007046 0.09624451 0.26636583 0.01937793
 0.02886832 0.05816001 0.01431296 0.2216506  0.02060988]
P(Y=1|texto) con Y=1 -> Positivo [0.32653633]
Distribución de los temas [0.22466208 0.03179353 0.0298572  0.02127031 0.1524412  0.02882699
 0.28675488 0.02556752 0.02129224 0.08592196 0.09161212]
P(Y=1|texto) con Y=1 -> Positivo [0.60610354]


<ipython-input-54-a0ab1eeee5ce>:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  doc_inst = modelo_final.make_doc(df_new['texto_procesado'][i])
